# Simulación del tráfico en un carril.

Primero se simulará para dos carros. La ecuación que simulará la velocidad de los vehículos es el mínimo entre

$$\dot{x_i} = \frac{1}{T_i}(x_{i-1}-x_i -\frac{l_i + l_{i-1}}{2}) 
$$ 

y

$$ \dot{x_i} = v_{max_{i}} $$

In [1]:
#Llamando al paquete de Taylor

using TaylorSeries

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 [inlined code] from /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl:127
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl, in expression starting on line 143
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 [inlined code] from /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl:127
 in anonymous at no file:0
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /home/windie/.julia/v0.4/IJulia/src/execute_request.jl:177
 in eventloop at /home/windie/.julia/v0.4/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading /home/windie/.julia/v0.4/TaylorSeries/src/Taylor1.jl, in expression starting on line 152
 in depwar

In [38]:
#Definiendo parámetros. Sólo para fijar ideas.

x1_0 = 0.5;
x2_0 = pi;
l1 = pi/16;
l2 = l1;
V1 = pi/4;
V2 = pi/2;



A continuación se escribe la función `modulo2pi` que encuentra la diferencia entre dos ángulos en módulo 2$\pi$

In [26]:
function modulo2pi(α,β)
    #Esta función calcula la diferencia entre dos ángulos y la devuelve 
    #módulo 2π.
    δ=α-β 
    
    if δ<0
        δ = 2π + δ
    else 
        δ = δ 
    end
    
    return δ
end

modulo2pi (generic function with 1 method)

In [29]:
#La siguiente función calula el mínimo entre la velocidad máxima y la velocidad de seguridad

function velocidad_sec{R<:Real }(T::R, car::Taylor1{R}, frontcar::Taylor1{R} , lfcar::R)
    vel = taylor1_variable(Float64, car.order)
     
    vel.coeffs[1] = 1/T * modulo2pi(frontcar.coeffs[1]-lfcar,car.coeffs[1])
    
    for i = 2 : car.order
        vel.coeffs[i] = 1/T * (frontcar.coeffs[i]-car.coeffs[i])
    end
        
    
    return vel
end

function velocidad_max(vmax)
    
    vel = vmax
    
    return vel
end


velocidad_max (generic function with 1 method)

Ahora se definen un par de variables tipo `Taylor1` que tendrá información de las series de taylor de las funciones $x_1 (t)$ y $x_2 (t)$.

In [15]:
x1 = taylor1_variable(Float64, 20);
x2 = taylor1_variable(Float64, 20);

Se le asignarán las condiciones iniciales a las funciones, esto es, los términos constantes en la serie. Los coeficientes de la parte lineal se hacen cero, porque empiezan como 1.

In [47]:
x1.coeffs[1] = x1_0;
x1.coeffs[2] = 0;
x2.coeffs[1] = x2_0;
x2.coeffs[2] = 0;

Ahora, a encontrar los siguientes coeficientes. Usando que $$ x_{n+1} = f_n /(n+1) $$

In [48]:
#Aquí se encuentran los coefficientes de x1 y x2 al rededor del cero.

for i = 2:20
    x1.coeffs[i] = velocidad_sec(1.0,x1,x2,π/50).coeffs[i-1]
    x2.coeffs[i] = velocidad_sec(1.0,x2,x1,π/50).coeffs[i-1]
end

In [49]:
x1

 0.5 + 2.578760800517997 t + 1.0 t² - 2.0 t³ + 4.0 t⁴ - 8.0 t⁵ + 16.0 t⁶ - 32.0 t⁷ + 64.0 t⁸ - 128.0 t⁹ + 256.0 t¹⁰ - 512.0 t¹¹ + 1024.0 t¹² - 2048.0 t¹³ + 4096.0 t¹⁴ - 8192.0 t¹⁵ + 16384.0 t¹⁶ - 32768.0 t¹⁷ + 65536.0 t¹⁸ - 131072.0 t¹⁹ + 𝒪(t²¹)

In [46]:
x2

 3.141592653589793 + 3.578760800517997 t - 1.0 t² + 2.0 t³ - 4.0 t⁴ + 8.0 t⁵ - 16.0 t⁶ + 32.0 t⁷ - 64.0 t⁸ + 128.0 t⁹ - 256.0 t¹⁰ + 512.0 t¹¹ - 1024.0 t¹² + 2048.0 t¹³ - 4096.0 t¹⁴ + 8192.0 t¹⁵ - 16384.0 t¹⁶ + 32768.0 t¹⁷ - 65536.0 t¹⁸ + 131072.0 t¹⁹ + 𝒪(t²¹)

In [53]:
evaluate(x1, 1)

-87377.92123919948